In [1]:
import PETsARD
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')


In [ ]:
loader = PETsARD.Loader(
    filepath='../[sunset]/data/[Adt Income] adult.csv',
    na_values={k: '?' for k in [
        'workclass',
        'occupation',
        'native-country'
    ]}
)
print(loader.data.head(1))


   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [ ]:
splitter = PETsARD.Splitter(
    data=loader.data,
    num_samples=1,  # 30,
    train_split_ratio=0.8
)
print(splitter.data[1]['train'].shape[0])
print(splitter.data[1]['validation'].shape[0])
print(splitter.data[1]['train'].head(1))
print(splitter.data[1]['validation'].head(1))


39073
9769
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
   age  workclass  fnlwgt   education  educational-num      marital-status  \
0   28  Local-gov  336951  Assoc-acdm               12  Married-civ-spouse   

        occupation relationship   race gender  capital-gain  capital-loss  \
0  Protective-serv      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              40  United-States   >50K  


In [ ]:
preproc = PETsARD.Preprocessor(
    data=splitter.data[1]['train'],
    missing=True,
    missing_method='drop',
    missing_columns=None,
    outlier=True,
    outlier_method='IQR',
    outlier_columns={'ignore': ['hours-per-week']},
    encoding=True,
    encoding_method='Label',
    encoding_columns=None,
    scaling=True,
    scaling_method='Standard',
    scaling_columns={'focus': 'fnlwgt'}
)
print('\n\n\n')
print(preproc.data.head(1))
print('\n\n\n')
print(preproc.encoder)
print('\n\n\n')
print(preproc.scaler)


Preprocessor - Outlierist (IQR): Dropped  1695 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   221 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped   231 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped  1070 rows on fnlwgt         . Kept [-62464.125, 416666.875] only.
Preprocessor - Outlierist (IQR): Dropped  2984 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  5995 in 36158 rows.
Preprocessor - Encoder (Label): Column marital-status  been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column gender          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column native-country  been labelized from 0 to 39.
Preprocessor - Encoder (Label): Column education       been labelized from 0 to 13.
Preprocessor - Encoder (Label): Column race            been labelized from 0 to  4.
Preprocessor - Encoder 

In [ ]:
synthesizer = PETsARD.Synthesizer(
    data=preproc.data,
    synthesizing_method='sdv-singletable-gaussiancoupula'
)
synthesizer.fit_sample()
print(synthesizer.data_syn.head(1))


ValueError: Synthesizer (SDV - SDV_SingleTableFactory): synthesizing_method sdv-singletable-gaussiancoupula didn't support.

In [6]:
# File D:\Dropbox\89_其他應用\GitHub\PETs-Experiment\PETsARD\Preprocessor\Missingist.py:27, in Missingist.inverse_transform(self, col_data)
#      25 _col_data = col_data.copy()
#      26 print(_col_data)
# ---> 27 _col_data[_na_mask] = np.nan
#      28 return _col_data

# ValueError: cannot convert float NaN to integer

postproc = PETsARD.Postprocessor(
    data=synthesizer.data_syn,
  # missingist = getattr(preproc ,'missingist' ,None),
    encoder=getattr(preproc, 'encoder', None),
    scaler=getattr(preproc, 'scaler', None)
)
print(postproc.data.head(1))


Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
   age workclass         fnlwgt education  educational-num marital-status  \
0   58   Private  189549.261742   HS-grad               10       Divorced   

       occupation relationship   race gender  capital-gain  capital-loss  \
0  Prof-specialty    Unmarried  White   Male             0             0   

   hours-per-week native-country income  
0              47     Yugoslavia  <=50K  


In [7]:
eval = PETsARD.Evaluator(evaluating_method='anonymeter-singlingout-univariate', data={'ori': splitter.data[1]['train'], 'syn': postproc.data, 'control': splitter.data[1]['validation']
                                                                                      }, anonymeter_n_attacks=2000
                         )
eval.eval()
eval.Evaluator.evaluation


Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 21.8733 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


{'Risk': 0.0,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.0013568577126237004,
 'Attack_Rate': 0.0009585236406264672,
 'Attack_Rate_err': 0.0009585236406264671,
 'Baseline_Rate': 0.0009585236406264672,
 'Baseline_Rate_err': 0.0009585236406264671,
 'Control_Rate': 0.0009585236406264672,
 'Control_Rate_err': 0.0009585236406264671}

In [8]:
eval = PETsARD.Evaluator(evaluating_method='anonymeter-linkability', data={'ori': splitter.data[1]['train'], 'syn': postproc.data, 'control': splitter.data[1]['validation']
                                                                           }, anonymeter_n_attacks=2000, anonymeter_n_neighbors=10, anonymeter_n_jobs=-1, anonymeter_aux_cols=[['age', 'fnlwgt', 'race', 'gender', 'native-country'], ['workclass', 'education', 'capital-gain', 'capital-loss', 'hours-per-week']
                                                                                                                                                                               ]
                         )
eval.eval()
eval.Evaluator.evaluation


Evaluator (Anonymeter - Linkability): Now is Linkability Evaluator
Evaluator (Anonymeter - Linkability): aux_cols are [age, fnlwgt, race, gender, native-country]
Evaluator (Anonymeter - Linkability): and [workclass, education, capital-gain, capital-loss, hours-per-week].
Evaluator (Anonymeter - Linkability): Evaluator time: 0.0003 sec.
Evaluator (Anonymeter): Evaluating  Linkability.
Evaluator (Anonymeter): Evaluating  Linkability spent 5.1737 sec.


{'Risk': 0.00550021800276215,
 'Risk_CI_btm': 0.001483115632946723,
 'Risk_CI_top': 0.009517320372577577,
 'Attack_Rate': 0.007446062833298323,
 'Attack_Rate_err': 0.0036434663392241715,
 'Baseline_Rate': 0.004950855451501456,
 'Baseline_Rate_err': 0.0029226088445362865,
 'Control_Rate': 0.001956606593345214,
 'Control_Rate_err': 0.00168232767569318}

In [9]:
eval = PETsARD.Evaluator(evaluating_method='anonymeter-inference', data={'ori': splitter.data[1]['train'], 'syn': postproc.data, 'control': splitter.data[1]['validation']
                                                                         }, anonymeter_n_attacks=2000, anonymeter_n_jobs=-1, anonymeter_secret='age'
                         )
eval.eval()
eval.Evaluator.evaluation


Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0002 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 0.5823 sec.


{'Risk': 0.023813609781145287,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.062409664673618484,
 'Attack_Rate': 0.1050344875036978,
 'Attack_Rate_err': 0.026600135400528267,
 'Baseline_Rate': 0.06335470980308298,
 'Baseline_Rate_err': 0.02100635332213679,
 'Control_Rate': 0.08320222299385194,
 'Control_Rate_err': 0.023904093583999354}

In [8]:
import PETsARD
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')


para_Executor = {
    'Loader_filepath': '../[sunset]/data/[Adt Income] adult.csv', 'Loader_na_values': {k: '?' for k in ['workclass', 'occupation', 'native-country']}, 'Splitter_num_samples': 2, 'Splitter_train_split_ratio': 0.8, 'Preprocessor_missing_method': 'drop', 'Preprocessor_outlier_method': 'IQR', 'Preprocessor_outlier_columns': {'ignore': ['hours-per-week']}, 'Preprocessor_encoding_method': 'Label', 'Preprocessor_scaling': True, 'Preprocessor_scaling_method': 'Standard', 'Preprocessor_scaling_columns': {'focus': 'fnlwgt'}, 'Synthesizer_synthesizing_method': 'sdv-singletable-gaussiancoupula', 'Evaluator_evaluating_method': 'anonymeter-singlingout-univariate'  # 'anonymeter-inference' #
    , 'Evaluator_anonymeter_n_attacks': 1  # 2000
    , 'Evaluator_num_samples': 2
}

executor_single = PETsARD.Executor(**para_Executor)
executor_single.run()
executor_single.evaluator[('[Adt Income] adult.csv',
                           '0.8x2',
                           1,
                           'drop-IQR-Label-Standard',
                           'sdv-singletable-gaussiancoupula',
                           'anonymeter-singlingout-univariatex2',
                           1)].Evaluator.evaluation

# executor_single = PETsARD.Executor(**para_Executor)
# executor_single.run_parallel()
# executor_single.evaluator[('[Adt Income] adult.csv',
#   '0.8x2',
#   1,
#   'drop-IQR-Label-Standard',
#   'sdv-singletable-gaussiancoupula',
#   'anonymeter-singlingout-univariatex2',
#   1)].Evaluator.evaluation


Executor - Loader: [Adt Income] adult.csv loading time: 10.104 sec.
Executor - Splitter: 0.8x2 splitting time: 0.096 sec.
Preprocessor - Outlierist (IQR): Dropped   208 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  3021 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   235 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped  1060 rows on fnlwgt         . Kept [-64359.75, 419126.25] only.
Preprocessor - Outlierist (IQR): Dropped  1696 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6026 in 36128 rows.
Preprocessor - Encoder (Label): Column education       been labelized from 0 to 13.
Preprocessor - Encoder (Label): Column income          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column gender          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column marital-status  been labelized f

ImportError: cannot import name '__version__' from 'PETsARD' (D:\Dropbox\89_其他應用\GitHub\PETsARD\PETsARD\__init__.py)

In [11]:
import PETsARD
import os
import sys

# sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
# os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')


para_Executor = {
    'Loader_filepath': '../[sunset]/data/[Adt Income] adult.csv', 'Loader_na_values': {k: '?' for k in ['workclass', 'occupation', 'native-country']}, 'Splitter_num_samples': 2, 'Splitter_train_split_ratio': 0.8, 'Preprocessor_missing_method': 'drop', 'Preprocessor_outlier_method': 'IQR', 'Preprocessor_outlier_columns': {'ignore': ['hours-per-week']}, 'Preprocessor_encoding_method': 'Label', 'Preprocessor_scaling': True, 'Preprocessor_scaling_method': 'Standard', 'Preprocessor_scaling_columns': {'focus': 'fnlwgt'}, 'Synthesizer_synthesizing_method': 'sdv-singletable-gaussiancoupula', 'Evaluator_evaluating_method': 'anonymeter-inference', 'Evaluator_anonymeter_n_attacks': 1  # 2000
    , 'Evaluator_num_samples': 2
}

executor_single = PETsARD.Executor(**para_Executor)
executor_single.run()
next(iter(
    executor_single.evaluator[('[Adt Income] adult.csv',
                               '0.8x2',
                               1,
                               'drop-IQR-Label-Standard',
                               'sdv-singletable-gaussiancoupula',
                               'anonymeter-inferencex2',
                               1)].values())).Evaluator.evaluation

# executor_single = PETsARD.Executor(**para_Executor)
# executor_single.run_parallel()
# next(iter(
#     executor_single.evaluator[('[Adt Income] adult.csv',
#         '0.8x2',
#         1,
#         'drop-IQR-Label-Standard',
#         'sdv-singletable-gaussiancoupula',
#         'anonymeter-inferencex2',
#         1)].values())).Evaluator.evaluation


Executor - Loader: [Adt Income] adult.csv loading time: 6.7149 sec.
Executor - Splitter: 0.8x2 splitting time: 0.0687 sec.
Preprocessor - Outlierist (IQR): Dropped  1061 rows on fnlwgt         . Kept [-64281.5, 419578.5] only.
Preprocessor - Outlierist (IQR): Dropped   229 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped  1711 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   227 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  3060 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6075 in 36134 rows.
Preprocessor - Encoder (Label): Column native-country  been labelized from 0 to 39.
Preprocessor - Encoder (Label): Column gender          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column race            been labelized from 0 to  4.
Preprocessor - Encoder (Label): Column relationship    been labelized fr

{'Risk': 0.0,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.03551128508197159,
 'Attack_Rate': 0.08518697431292883,
 'Attack_Rate_err': 0.024167884844459902,
 'Baseline_Rate': 0.07923272035569816,
 'Baseline_Rate_err': 0.02336369331827033,
 'Control_Rate': 0.08717172563200573,
 'Control_Rate_err': 0.02442758882817536}

In [ ]:
import pandas as pd
for i in ['01', '02', '03']:  # ,'04','05','06'
    eval = PETsARD.Evaluator(evaluating_method='anonymeter-singlingout-univariate', data={'ori': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv"), 'syn': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}-1-1]Postproc.csv"), 'control': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ctrl].csv")
                                                                                          }, anonymeter_n_attacks=500
                             )
    eval.eval()
    print(eval.Evaluator.evaluation)


In [ ]:
import pandas as pd


def Result(__evaluator):
    import numpy as np
    __dict_result = {}
    __para_to_handle = [('Risk', ['risk()', 'value']), ('Risk_CI_btm', ['risk()', 'ci[0]']), ('Risk_CI_top', ['risk()', 'ci[1]']), ('Attack_Rate', ['results()', 'attack_rate', 'value']), ('Attack_Rate_err', ['results()', 'attack_rate', 'error']), ('Baseline_Rate', ['results()', 'baseline_rate', 'value']), ('Baseline_Rate_err', ['results()', 'baseline_rate', 'error']), ('Control_Rate', ['results()', 'control_rate', 'value']), ('Control_Rate_err', ['results()', 'control_rate', 'error'])
                        ]
    for __key, __attrs in __para_to_handle:
        try:
            __attr_value = __evaluator
            for __attr in __attrs:
                if '()' in __attr:
                    __method_name = __attr.split('(')[0]
                    if hasattr(__attr_value, __method_name):
                        __method = getattr(__attr_value, __method_name)
                        if callable(__method):
                            __attr_value = __method()
                        else:
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __dict_result[__key] = np.nan
                        break
                elif '[' in __attr:
                    __attr_name = __attr.split('[')[0]
                    __index = int(__attr.split('[')[1].rstrip(']'))
                    if hasattr(__attr_value, __attr_name)\
                            and isinstance(getattr(__attr_value, __attr_name), (list, dict, tuple)):
                        try:
                            __attr_value = getattr(
                                __attr_value, __attr_name)[__index]
                        except (IndexError, KeyError):
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __dict_result[__key] = np.nan
                        break
                else:
                    __attr_value = getattr(__attr_value, __attr)
            __dict_result[__key] = __attr_value
        except Exception as e:
            __dict_result[__key] = np.nan
    return __dict_result


for i in ['01', '02', '03']:  # ,'04','05','06'
    from anonymeter.evaluators import SinglingOutEvaluator
    print(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv")
    evaluator = SinglingOutEvaluator(ori=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv"), syn=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}-1-1]Postproc.csv"), control=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ctrl].csv"), n_attacks=2000
                                     )
    try:
        evaluator.evaluate(mode='univariate')
        print(Result(evaluator))
    except RuntimeError as ex:
        print(f"Singling out evaluation failed with {ex}."
              "Please re-run this cell."
              "For more stable results increase `n_attacks`. Note that this will "
              "make the evaluation slower.")


In [ ]:
import pandas as pd
for i in ['01', '02', '03']:  # ,'04','05','06'
    eval = PETsARD.Evaluator(evaluating_method='anonymeter-singlingout-univariate', data={'ori': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv"), 'syn': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}-1-1]Postproc.csv"), 'control': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ctrl].csv")
                                                                                          }, anonymeter_n_attacks=500
                             )
    eval.eval()
    print(eval.Evaluator.evaluation)


In [ ]:
import itertools
for type in ['][Ori]', '][Ctrl]', '-1-1]Postproc']:
    for combo in itertools.combinations(['01', '02', '03', '04', '05', '06'], 2):
        df_a = pd.read_csv(
            f"PETsARD[20231224-085805]_Trial[{combo[0]}{type}.csv")
        df_b = pd.read_csv(
            f"PETsARD[20231224-085805]_Trial[{combo[1]}{type}.csv")
        if df_a.equals(df_b):
            print(type+': '+str(combo))
            print("They're same!!??")
        # else:
            # print(type+': '+str(combo))
            # print("They're different.")
print('done.')


In [ ]:
import PETsARD
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')


para_Executor = {
    'Loader_filepath': '../[sunset]/data/[Adt Income] adult.csv', 'Loader_na_values': {k: '?' for k in ['workclass', 'occupation', 'native-country']}, 'Splitter_num_samples': 2, 'Splitter_train_split_ratio': 0.8, 'Preprocessor_missing_method': 'drop', 'Preprocessor_outlier_method': 'IQR', 'Preprocessor_outlier_columns': {'ignore': ['hours-per-week']}, 'Preprocessor_encoding_method': 'Label', 'Preprocessor_scaling': True, 'Preprocessor_scaling_method': 'Standard', 'Preprocessor_scaling_columns': {'focus': 'fnlwgt'}, 'Synthesizer_synthesizing_method': 'sdv-singletable-gaussiancoupula', 'Evaluator_evaluating_method': 'anonymeter-inference', 'Evaluator_anonymeter_n_attacks': 1  # 2000
    , 'Evaluator_num_samples': 2
}

executor_single = PETsARD.Executor(**para_Executor)
executor_single.run_single_process()
next(iter(
    executor_single.evaluator[('[Adt Income] adult.csv',
                               '0.8',
                               1,
                               'drop-IQR-Label-Standard',
                               'sdv-singletable-gaussiancoupula',
                               'anonymeter-inference',
                               1)].values())).Evaluator.evaluation

executor_single = PETsARD.Executor(**para_Executor)
executor_single.run()
next(iter(
    executor_single.evaluator[('[Adt Income] adult.csv',
                               '0.8',
                               1,
                               'drop-IQR-Label-Standard',
                               'sdv-singletable-gaussiancoupula',
                               'anonymeter-inference',
                               1)].values())).Evaluator.evaluation


{'Risk': 0.0022375332675201006,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.046174667591818215,
 'Attack_Rate': 0.11495824409908227,
 'Attack_Rate_err': 0.027695443904980018,
 'Baseline_Rate': 0.07724796903662126,
 'Baseline_Rate_err': 0.023086784285609292,
 'Control_Rate': 0.11297349278000537,
 'Control_Rate_err': 0.027482077623402282}

In [19]:
import pandas as pd


def Result(__evaluator):
    import numpy as np
    __dict_result = {}
    __para_to_handle = [('Risk', ['risk()', 'value']), ('Risk_CI_btm', ['risk()', 'ci[0]']), ('Risk_CI_top', ['risk()', 'ci[1]']), ('Attack_Rate', ['results()', 'attack_rate', 'value']), ('Attack_Rate_err', ['results()', 'attack_rate', 'error']), ('Baseline_Rate', ['results()', 'baseline_rate', 'value']), ('Baseline_Rate_err', ['results()', 'baseline_rate', 'error']), ('Control_Rate', ['results()', 'control_rate', 'value']), ('Control_Rate_err', ['results()', 'control_rate', 'error'])
                        ]
    for __key, __attrs in __para_to_handle:
        try:
            __attr_value = __evaluator
            for __attr in __attrs:
                if '()' in __attr:
                    __method_name = __attr.split('(')[0]
                    if hasattr(__attr_value, __method_name):
                        __method = getattr(__attr_value, __method_name)
                        if callable(__method):
                            __attr_value = __method()
                        else:
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __dict_result[__key] = np.nan
                        break
                elif '[' in __attr:
                    __attr_name = __attr.split('[')[0]
                    __index = int(__attr.split('[')[1].rstrip(']'))
                    if hasattr(__attr_value, __attr_name)\
                            and isinstance(getattr(__attr_value, __attr_name), (list, dict, tuple)):
                        try:
                            __attr_value = getattr(
                                __attr_value, __attr_name)[__index]
                        except (IndexError, KeyError):
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __dict_result[__key] = np.nan
                        break
                else:
                    __attr_value = getattr(__attr_value, __attr)
            __dict_result[__key] = __attr_value
        except Exception as e:
            __dict_result[__key] = np.nan
    return __dict_result


for i in ['01', '02', '03']:  # ,'04','05','06'
    from anonymeter.evaluators import SinglingOutEvaluator
    print(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv")
    evaluator = SinglingOutEvaluator(ori=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv"), syn=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}-1-1]Postproc.csv"), control=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ctrl].csv"), n_attacks=2000
                                     )
    try:
        evaluator.evaluate(mode='univariate')
        print(Result(evaluator))
    except RuntimeError as ex:
        print(f"Singling out evaluation failed with {ex}."
              "Please re-run this cell."
              "For more stable results increase `n_attacks`. Note that this will "
              "make the evaluation slower.")


Executor - Loader: adult loading time: 6.8097 sec.
Executor - Splitter: 0.8 splitting time: 0.339 sec.
Preprocessor - Outlierist (IQR): Dropped  1060 rows on fnlwgt         . Kept [-63981.5, 419234.5] only.
Preprocessor - Outlierist (IQR): Dropped   227 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped  1705 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  9432 rows on hours-per-week . Kept [32.5, 52.5] only.
Preprocessor - Outlierist (IQR): Dropped   214 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  3030 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped 13932 in 36207 rows.
Preprocessor - Encoder (Label): Column native-country  been labelized from 0 to 39.
Preprocessor - Encoder (Label): Column gender          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column race            been labelized from 0 to 

Found 765 failed queries out of 2000. Check DEBUG messages for more details.


Executor - Evaluator: anonymeter-SinglingOut at 1 trials evaluating time: 131.365 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0322 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


Found 802 failed queries out of 2000. Check DEBUG messages for more details.


Executor - Evaluator: anonymeter-SinglingOut at 2 trials evaluating time: 131.1331 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0336 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


...

Found 821 failed queries out of 2000. Check DEBUG messages for more details.


Executor - Evaluator: anonymeter-SinglingOut at 5 trials evaluating time: 132.587 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.036 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


Found 800 failed queries out of 2000. Check DEBUG messages for more details.


Executor - Evaluator: anonymeter-SinglingOut at 6 trials evaluating time: 131.8783 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0352 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


Found 799 failed queries out of 2000. Check DEBUG messages for more details.


In [ ]:
import pandas as pd
for i in ['01', '02', '03']:  # ,'04','05','06'
    eval = PETsARD.Evaluator(evaluating_method='anonymeter-singlingout-univariate', data={'ori': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv"), 'syn': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}-1-1]Postproc.csv"), 'control': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ctrl].csv")
                                                                                          }, anonymeter_n_attacks=500
                             )
    eval.eval()
    print(eval.Evaluator.evaluation)


In [18]:
import itertools
for type in ['][Ori]', '][Ctrl]', '-1-1]Postproc']:
    for combo in itertools.combinations(['01', '02', '03', '04', '05', '06'], 2):
        df_a = pd.read_csv(
            f"PETsARD[20231224-085805]_Trial[{combo[0]}{type}.csv")
        df_b = pd.read_csv(
            f"PETsARD[20231224-085805]_Trial[{combo[1]}{type}.csv")
        if df_a.equals(df_b):
            print(type+': '+str(combo))
            print("They're same!!??")
        # else:
            # print(type+': '+str(combo))
            # print("They're different.")
print('done.')


Executor - Evaluator: anonymeter-SinglingOut at 7 trials evaluating time: 131.5421 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0354 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


In [1]:
import PETsARD
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')


para_Executor = {
    'Loader': {
        'adult': {'filepath': '../[sunset]/data/[Adt Income] adult.csv', 'na_values': {k: '?' for k in ['workclass', 'occupation', 'native-country']}
                  }
        # ,'NHANES' : {'filepath'     : '../[sunset]/data/[NHANES] B.csv'
        #             ,'header_exist' : False
        #             ,'header_names' : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
        # }
    }, 'Splitter': {
        '0.8': {'num_samples': 2  # 10
                , 'train_split_ratio': 0.8
                }
        # ,'0.05' : {'num_samples'       : 10
        #           ,'train_split_ratio' : 0.05
        # }
    }, 'Preprocessor': {
        'drop-IQR-stanard-NA': {'missing_method': 'drop', 'outlier_method': 'IQR'                                # ,'outlier_columns' : {'ignore': ['hours-per-week']}
                                , 'encoding_method': 'Label', 'scaling': False
                                # ,'scaling_method'  : 'Standard'
                                #   ,'scaling_columns' : {'focus' : 'fnlwgt'}
                                }
        # ,'drop-IQR-minmax-NA' : {'missing_method'  : 'drop'
        #           ,'outlier_method'  : 'IQR'
        #           # ,'outlier_columns' : {'ignore': ['hours-per-week']}
        #           ,'encoding_method' : 'Label'
        #           ,'scaling'         : False
        #           # ,'scaling_method'  : 'minmax'
        #         #   ,'scaling_columns' : {'focus' : 'fnlwgt'}
        # }
    }, 'Synthesizer': {
        'GaussianCoupula': {'synthesizing_method': 'sdv-singletable-gaussiancoupula'}
        #    ,'TVAE'            : {'synthesizing_method' : 'sdv-singletable-tvae'}
        #    ,'CoupulaGAN'      : {'synthesizing_method' : 'sdv-singletable-coupulagan'}
        #    ,'CTGAN'           : {'synthesizing_method' : 'sdv-singletable-ctgan'}
    }, 'Evaluator': {
        'anonymeter-SinglingOut': {'evaluating_method': 'anonymeter-singlingout-univariate', 'num_samples': 2  # 10
                                   , 'anonymeter_n_attacks': 1  # 2000
                                   # 10
                                   # 2000
                                   }, 'anonymeter-Linkability': {'evaluating_method': 'anonymeter-linkability', 'num_samples': 2, 'anonymeter_n_attacks': 1, 'anonymeter_n_neighbors': 5, 'anonymeter_aux_cols': [['age', 'fnlwgt', 'race', 'gender', 'native-country'], ['workclass', 'education', 'capital-gain', 'capital-loss', 'hours-per-week']
                                                                                                                                                                                                                  ]
                                                                 },
        'anonymeter-Inference': {'evaluating_method': 'anonymeter-inference', 'num_samples': 2  # 10
                                 , 'anonymeter_n_attacks': 1  # 2000
                                 , 'anonymeter_secrets': True
                                 }
    }
}

executor_adult = PETsARD.Executor(**para_Executor)
# executor_adult.run()
executor_adult.run_parallel()


Loading: 100%|██████████| 1/1 [00:13<00:00, 13.86s/it]

Synthesizing:   0%|          | 0/2 [00:00<?, ?it/s]


Synthesizer (SDV - SingleTable): Metafile loading time: 0.0595 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.1476 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 14.1998 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 14.2393 sec.


Synthesizing:  50%|█████     | 1/2 [00:15<00:15, 15.47s/it]

Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 22187 rows (same as raw) in 1.0931 sec.
Executor - Synthesizer: GaussianCoupula synthesizing time: 15.3715 sec.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Executor - Postprocessor: postprocessing time: 0.0793 sec.


Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 22187 rows (same as raw) in 1.1964 sec.
Executor - Synthesizer: GaussianCoupula synthesizing time: 15.6023 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.9176 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


...

Evaluator (Anonymeter): Evaluating  Linkability spent 0.2184 sec.
Executor - Evaluator: anonymeter-Linkability at 1 trials evaluating time: 0.2285 sec.
Evaluator (Anonymeter - Linkability): Now is Linkability Evaluator
Evaluator (Anonymeter - Linkability): aux_cols are [age, fnlwgt, race, gender, native-country]
Evaluator (Anonymeter - Linkability): and [workclass, education, capital-gain, capital-loss, hours-per-week].
Evaluator (Anonymeter - Linkability): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Linkability.


Evaluator (Anonymeter): Evaluating  Linkability spent 0.3247 sec.
Executor - Evaluator: anonymeter-Linkability at 2 trials evaluating time: 0.3322 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 0.6198 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 0.6752 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 0.629 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymete

Evaluator (Anonymeter): Evaluating  Inference spent 0.6024 sec.
Executor - Evaluator: anonymeter-Inference at 1 trials evaluating time: 9.4673 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 0.6499 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 0.614 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 0.6436 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): 


Synthesizing: 100%|██████████| 2/2 [01:06<00:00, 33.27s/it]

Evaluator (Anonymeter): Evaluating  Inference spent 0.6367 sec.
Executor - Evaluator: anonymeter-Inference at 2 trials evaluating time: 9.6876 sec.
====== ====== ====== ====== ====== ======
Executor (run - parallel): Total execution time: 86.7449 sec.
====== ====== ====== ====== ====== ======


In [ ]:
import PETsARD
import os
import sys

# sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
# os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')


para_Executor = {
    'Loader': {
        #  'adult'  : {'filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
        #             ,'na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
        # }
        , 'NHANES': {'filepath': '../[sunset]/data/[NHANES] B.csv', 'header_exist': False, 'header_names': ['gen', 'age', 'race', 'edu', 'mar', 'bmi', 'dep', 'pir', 'gh', 'mets', 'qm', 'dia']
                     }
    }, 'Splitter': {
        '0.8': {'num_samples': 10, 'train_split_ratio': 0.8
                }
        # ,'0.05' : {'num_samples'       : 10
        #           ,'train_split_ratio' : 0.05
        # }
    }, 'Preprocessor': {
        'drop-IQR-stanard-NA': {'missing_method': 'drop', 'outlier_method': 'IQR'                                # ,'outlier_columns' : {'ignore': ['hours-per-week']}
                                , 'encoding_method': 'Label', 'scaling': False
                                # ,'scaling_method'  : 'Standard'
                                #   ,'scaling_columns' : {'focus' : 'fnlwgt'}
                                }
        # ,'drop-IQR-minmax-NA' : {'missing_method'  : 'drop'
        #           ,'outlier_method'  : 'IQR'
        #           # ,'outlier_columns' : {'ignore': ['hours-per-week']}
        #           ,'encoding_method' : 'Label'
        #           ,'scaling'         : False
        #           # ,'scaling_method'  : 'minmax'
        #         #   ,'scaling_columns' : {'focus' : 'fnlwgt'}
        # }
    }, 'Synthesizer': {
        'GaussianCoupula': {'synthesizing_method': 'sdv-singletable-gaussiancoupula'}, 'TVAE': {'synthesizing_method': 'sdv-singletable-tvae'}, 'CoupulaGAN': {'synthesizing_method': 'sdv-singletable-coupulagan'}, 'CTGAN': {'synthesizing_method': 'sdv-singletable-ctgan'}
    }, 'Evaluator': {
        'anonymeter-SinglingOut': {'evaluating_method': 'anonymeter-singlingout-univariate', 'num_samples': 10, 'anonymeter_n_attacks': 2000
                                   }, 'anonymeter-Linkability': {'evaluating_method': 'anonymeter-linkability', 'num_samples': 10, 'anonymeter_n_attacks': 2000, 'anonymeter_n_neighbors': 5, 'anonymeter_aux_cols': [['age', 'fnlwgt', 'race', 'gender', 'native-country'], ['workclass', 'education', 'capital-gain', 'capital-loss', 'hours-per-week']
                                                                                                                                                                                                                      ]
                                                                 }, 'anonymeter-Inference': {'evaluating_method': 'anonymeter-inference', 'num_samples': 10, 'anonymeter_n_attacks': 2000, 'anonymeter_secrets': True
                                                                                             }
    }, 'Evaluator': {
        'SinglingOut': {'method': 'anonymeter-singlingout-univariate', 'num_samples': 2  # 30
                        , 'anonymeter_n_attacks': 2  # 2000
                        # 30
                        # 2000
                        }, 'Linkability': {'method': 'anonymeter-linkability', 'num_samples': 2, 'anonymeter_n_attacks': 2, 'anonymeter_n_neighbors': 10, 'anonymeter_aux_cols': [['age', 'fnlwgt', 'race', 'gender', 'native-country'], ['workclass', 'education', 'capital-gain', 'capital-loss', 'hours-per-week']
                                                                                                                                                                                  ]
                                           # 30
                                           # 2000
                                           }, 'Inference': {'method': 'anonymeter-inference', 'num_samples': 2, 'anonymeter_n_attacks': 2, 'anonymeter_secrets': True
                                                            }
    }
}

executor_ = PETsARD.Executor(**para_Executor)
executor_NHANES.run()
# executor_NHANES.run_single_process()
